In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import re

In [2]:
movies_data = pd.read_csv('movies.csv')

In [3]:
books_data = pd.read_csv('books.csv')

In [4]:
movies_data['genres'] = movies_data['genres'].fillna('')
books_data['categories'] = books_data['categories'].fillna('')

In [5]:
tfidf_vectorizer_movies = TfidfVectorizer(stop_words='english')
tfidf_matrix_movies = tfidf_vectorizer_movies.fit_transform(movies_data['genres'])

In [6]:
tfidf_vectorizer_books = TfidfVectorizer(stop_words='english')
tfidf_matrix_books = tfidf_vectorizer_books.fit_transform(books_data['categories'])

In [7]:
cosine_sim_movies = linear_kernel(tfidf_matrix_movies, tfidf_matrix_movies)
cosine_sim_books = linear_kernel(tfidf_matrix_books, tfidf_matrix_books)

In [8]:
def recommend_movies(movie_title, cosine_sim=cosine_sim_movies):
    pattern = re.compile(movie_title, re.IGNORECASE)
    similar_titles = movies_data[movies_data['title'].apply(lambda x: bool(pattern.search(x)))]
    similar_indices = similar_titles.index.tolist()
    recommendations = []
    for idx in similar_indices:
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        movie_indices = [i[0] for i in sim_scores]
        recommendations.extend(movies_data['title'].iloc[movie_indices])
    return recommendations

In [9]:
def recommend_books(book_title, cosine_sim=cosine_sim_books):
    pattern = re.compile(book_title, re.IGNORECASE)
    similar_titles = books_data[books_data['title'].apply(lambda x: bool(pattern.search(x)))]
    similar_indices = similar_titles.index.tolist()
    recommendations = []
    for idx in similar_indices:
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        book_indices = [i[0] for i in sim_scores]
        recommendations.extend(books_data['title'].iloc[book_indices])
    return recommendations

In [ ]:
search_type = input("What do you want to search for? Enter 'movies' or 'books': ").lower()
j=1
if search_type == 'movies':
    movie_title_pattern = input("Enter a part of the movie title: ")
    recommended_movies = recommend_movies(movie_title_pattern)
    print("\nRecommended movies similar to '{}' are:".format(movie_title_pattern))
    for i in recommended_movies:
        print("{} {}".format(j,i))
        j=j+1

elif search_type == 'books':
    book_title_pattern = input("Enter a part of the book title: ")
    recommended_books = recommend_books(book_title_pattern)
    print("\nRecommended books similar to '{}' are:".format(book_title_pattern))
    for i in recommended_books:
        print("{} {}".format(j,i))
        j=j+1

else:
    print("Invalid input. Please enter 'movies' or 'books'.")